# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory Residential Electric Heating

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ResElecElec/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ResElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ResElecElec/" + file)
    all_ResElec = pd.concat([all_ResElec, df])
    
# all_ResElec

# 105120  rows, 108 columns

In [10]:
# Get wanted columns
all_ResElec1 = all_ResElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

# all_ComElec1

In [11]:
all_ResElec1 = all_ResElec1.dropna().reset_index(drop=True)

# all_ResElec1

In [12]:
# # Export grouped file to csv
# all_ResElec1.to_csv("Scraper_Output/cle_res_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [13]:
# Rename Columns
all_ResElec2 = all_ResElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

# all_ResElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [14]:
# Copy dataframe to modify and leave original df intact
get_annual_ResElec = all_ResElec2.copy(deep=True)

# get_annual_ResElec

In [15]:
# Convert Joules to KBtu in dataframe
get_annual_ResElec["FanEnergy(kBtu)"] = get_annual_ResElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingElecEnergy(kBtu)"] = get_annual_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingGasEnergy(kBtu)"] = get_annual_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["CoolingEnergy(kBtu)"] = get_annual_ResElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ResElec["LightingEnergy(kBtu)"] = (get_annual_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux


# get_annual_ComElec

In [16]:
# Drop columns
get_annual_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ResElec

In [17]:
# Add energy totals
totalResEnergy = get_annual_ResElec["FanEnergy(kBtu)"] + get_annual_ResElec["HeatingElecEnergy(kBtu)"] + get_annual_ResElec["LightingEnergy(kBtu)"] + get_annual_ResElec["CoolingEnergy(kBtu)"]
        
get_annual_ResElec["AnnualEnergy(kBtu)"] = totalResEnergy

# get_annual_ResElec

-----

## Get Energy per sf 
Add EUI (site) column<br>

In [18]:
# Get data per sf (energy/sf)
get_annual_ResElec["FanEnergy(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ResElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ResElec["LightingEnergy(kBtu/sf)"] = (get_annual_ResElec["LightingEnergy(kBtu)"]) / sf

In [19]:
# Establish EUI (energy/sf)
get_annual_ResElec["EUI(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
                                                       
# get_annual_ResElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [20]:
# Do carbon calculation using variable (lb/sf)
get_annual_ResElec["CO2_Electric(lb/sf)"] = ((get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ResElec

In [21]:
get_annual_ResElec["CO2_Gas(lb/sf)"] = ((get_annual_ResElec["HeatingGasEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual_ComElec

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [22]:
# Get annual costs
get_annual_costs_ResElec = all_ResElec2.copy(deep=True)

In [23]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ResElec["FanCost($)"] = (get_annual_costs_ResElec["FanEnergy[J](Hourly)"] * .000000277777778) * reselec_cost
get_annual_costs_ResElec["HeatingElecCost($)"] = ((get_annual_costs_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["HeatingGasCost($)"] = ((get_annual_costs_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["CoolingCost($)"] = (get_annual_costs_ResElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * reselec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ResElec["LightingCost($)"] = (get_annual_costs_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

# get_annual_costs

In [24]:
# Drop columns
get_annual_costs_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

# get_annual_costs_ResElec

In [25]:
# Add cost totals
totalResCost = get_annual_costs_ResElec["FanCost($)"] + get_annual_costs_ResElec["HeatingElecCost($)"] + get_annual_costs_ResElec["HeatingGasCost($)"]  + get_annual_costs_ResElec["CoolingCost($)"] + get_annual_costs_ResElec["LightingCost($)"]
        
get_annual_costs_ResElec["AnnualCost($)"] = totalResCost

# get_annual_costs_ResElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>

In [26]:
# Get data per sf ($/sf)
get_annual_costs_ResElec["FanCost($/sf)"] = (get_annual_costs_ResElec["FanCost($)"]) / sf
get_annual_costs_ResElec["HeatingElecCost($/sf)"] = (get_annual_costs_ResElec["HeatingElecCost($)"]) / sf
get_annual_costs_ResElec["HeatingGasCost($/sf)"] = (get_annual_costs_ResElec["HeatingGasCost($)"]) / sf
get_annual_costs_ResElec["CoolingCost($/sf)"] = (get_annual_costs_ResElec["CoolingCost($)"]) / sf
get_annual_costs_ResElec["LightingCost($/sf)"] = (get_annual_costs_ResElec["LightingCost($)"]) / sf

# get_annual_costs_ResElec

In [27]:
# Establish costs ($/sf)
get_annual_costs_ResElec["AnnualCost($/sf)"] = (get_annual_costs_ResElec["FanCost($)"] +
                                    get_annual_costs_ResElec["HeatingElecCost($)"] +
                                    get_annual_costs_ResElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ResElec["CoolingCost($)"] +  
                                    get_annual_costs_ResElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ResElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [28]:
# Combine data for energy and cost
allAnnualDataResElec = pd.merge(get_annual_ResElec, get_annual_costs_ResElec)

# allAnnualDataResElec

In [29]:
# list(allAnnualDataResElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br> 
Export to file

In [30]:
# Group by scenario
annualDataRes = allAnnualDataResElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataRes.sum().round(2)

In [31]:
annualDataResFinal = annualDataRes.sum().round(2)

# annualDataResFinal

In [32]:
# Export grouped file to csv
annualDataResFinal.to_csv("Scraper_Output/cle_annual_res.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [33]:
# Define the scenario for the baselines.
baselineAnnualDataResElec = allAnnualDataResElec[allAnnualDataResElec["GlazingSystem"] == "Exist"]

baselineAnnualDataResElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),...,HeatingGasCost($),CoolingCost($),LightingCost($),AnnualCost($),FanCost($/sf),HeatingElecCost($/sf),HeatingGasCost($/sf),CoolingCost($/sf),LightingCost($/sf),AnnualCost($/sf)
26280,cle_res_elec_01,01/01 01:00:00,N,Exist,1.811737,4.531636,0.0,0.0,0.015075,6.358448,...,0.0,0.0,0.015075,0.133376,0.000199,0.000160,0.0,0.0,0.000046,0.000405
26281,cle_res_elec_01,01/01 02:00:00,N,Exist,1.811737,4.382968,0.0,0.0,0.015075,6.209780,...,0.0,0.0,0.015075,0.131651,0.000199,0.000154,0.0,0.0,0.000046,0.000399
26282,cle_res_elec_01,01/01 03:00:00,N,Exist,1.811737,4.527612,0.0,0.0,0.015075,6.354424,...,0.0,0.0,0.015075,0.133329,0.000199,0.000159,0.0,0.0,0.000046,0.000405
26283,cle_res_elec_01,01/01 04:00:00,N,Exist,1.811737,4.577138,0.0,0.0,0.015075,6.403950,...,0.0,0.0,0.015075,0.133903,0.000199,0.000161,0.0,0.0,0.000046,0.000406
26284,cle_res_elec_01,01/01 05:00:00,N,Exist,1.811737,4.600883,0.0,0.0,0.042074,6.454694,...,0.0,0.0,0.042074,0.161178,0.000199,0.000162,0.0,0.0,0.000128,0.000489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61315,cle_res_elec_04,12/31 20:00:00,W,Exist,1.462425,2.438205,0.0,0.0,0.221847,4.122477,...,0.0,0.0,0.221847,0.303190,0.000161,0.000086,0.0,0.0,0.000673,0.000920
61316,cle_res_elec_04,12/31 21:00:00,W,Exist,1.462425,2.603133,0.0,0.0,0.224997,4.290555,...,0.0,0.0,0.224997,0.308253,0.000161,0.000092,0.0,0.0,0.000683,0.000935
61317,cle_res_elec_04,12/31 22:00:00,W,Exist,1.462425,2.700461,0.0,0.0,0.155698,4.318584,...,0.0,0.0,0.155698,0.240083,0.000161,0.000095,0.0,0.0,0.000472,0.000729
61318,cle_res_elec_04,12/31 23:00:00,W,Exist,1.462425,2.803635,0.0,0.0,0.086399,4.352460,...,0.0,0.0,0.086399,0.171981,0.000161,0.000099,0.0,0.0,0.000262,0.000522


In [34]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualDataResElec = baselineAnnualDataResElec.rename(columns={"Scenario":"B_Scenario",
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'B_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'B_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'B_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'B_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingElecCost($)':'B_HeatingElecCost($)',
 'HeatingGasCost($)':'B_HeatingGasCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingElecCost($/sf)':'B_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'B_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataResElec

In [35]:
# Reset index
allBaselineAnnualDataResElec.reset_index(inplace=True, drop=True)

allBaselineAnnualDataResElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_FanEnergy(kBtu),B_HeatingElecEnergy(kBtu),B_HeatingGasEnergy(kBtu),B_CoolingEnergy(kBtu),B_LightingEnergy(kBtu),B_AnnualEnergy(kBtu),...,B_HeatingGasCost($),B_CoolingCost($),B_LightingCost($),B_AnnualCost($),B_FanCost($/sf),B_HeatingElecCost($/sf),B_HeatingGasCost($/sf),B_CoolingCost($/sf),B_LightingCost($/sf),B_AnnualCost($/sf)
0,cle_res_elec_01,01/01 01:00:00,N,Exist,1.811737,4.531636,0.0,0.0,0.015075,6.358448,...,0.0,0.0,0.015075,0.133376,0.000199,0.000160,0.0,0.0,0.000046,0.000405
1,cle_res_elec_01,01/01 02:00:00,N,Exist,1.811737,4.382968,0.0,0.0,0.015075,6.209780,...,0.0,0.0,0.015075,0.131651,0.000199,0.000154,0.0,0.0,0.000046,0.000399
2,cle_res_elec_01,01/01 03:00:00,N,Exist,1.811737,4.527612,0.0,0.0,0.015075,6.354424,...,0.0,0.0,0.015075,0.133329,0.000199,0.000159,0.0,0.0,0.000046,0.000405
3,cle_res_elec_01,01/01 04:00:00,N,Exist,1.811737,4.577138,0.0,0.0,0.015075,6.403950,...,0.0,0.0,0.015075,0.133903,0.000199,0.000161,0.0,0.0,0.000046,0.000406
4,cle_res_elec_01,01/01 05:00:00,N,Exist,1.811737,4.600883,0.0,0.0,0.042074,6.454694,...,0.0,0.0,0.042074,0.161178,0.000199,0.000162,0.0,0.0,0.000128,0.000489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,cle_res_elec_04,12/31 20:00:00,W,Exist,1.462425,2.438205,0.0,0.0,0.221847,4.122477,...,0.0,0.0,0.221847,0.303190,0.000161,0.000086,0.0,0.0,0.000673,0.000920
35036,cle_res_elec_04,12/31 21:00:00,W,Exist,1.462425,2.603133,0.0,0.0,0.224997,4.290555,...,0.0,0.0,0.224997,0.308253,0.000161,0.000092,0.0,0.0,0.000683,0.000935
35037,cle_res_elec_04,12/31 22:00:00,W,Exist,1.462425,2.700461,0.0,0.0,0.155698,4.318584,...,0.0,0.0,0.155698,0.240083,0.000161,0.000095,0.0,0.0,0.000472,0.000729
35038,cle_res_elec_04,12/31 23:00:00,W,Exist,1.462425,2.803635,0.0,0.0,0.086399,4.352460,...,0.0,0.0,0.086399,0.171981,0.000161,0.000099,0.0,0.0,0.000262,0.000522


In [36]:
# Define the scenario for the baselines.

clearAnnualDataResElec = allAnnualDataResElec[allAnnualDataResElec["GlazingSystem"] == "Exist_Clear"]

clearAnnualDataResElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),...,HeatingGasCost($),CoolingCost($),LightingCost($),AnnualCost($),FanCost($/sf),HeatingElecCost($/sf),HeatingGasCost($/sf),CoolingCost($/sf),LightingCost($/sf),AnnualCost($/sf)
61320,cle_res_elec_05,01/01 01:00:00,N,Exist_Clear,1.620319,2.234077,0.0,0.0,0.015075,3.869470,...,0.0,0.0,0.015075,0.099779,0.000178,0.000079,0.0,0.0,0.000046,0.000303
61321,cle_res_elec_05,01/01 02:00:00,N,Exist_Clear,1.620319,2.196333,0.0,0.0,0.015075,3.831727,...,0.0,0.0,0.015075,0.099341,0.000178,0.000077,0.0,0.0,0.000046,0.000301
61322,cle_res_elec_05,01/01 03:00:00,N,Exist_Clear,1.620319,2.285303,0.0,0.0,0.015075,3.920696,...,0.0,0.0,0.015075,0.100373,0.000178,0.000080,0.0,0.0,0.000046,0.000305
61323,cle_res_elec_05,01/01 04:00:00,N,Exist_Clear,1.620319,2.338091,0.0,0.0,0.015075,3.973484,...,0.0,0.0,0.015075,0.100985,0.000178,0.000082,0.0,0.0,0.000046,0.000306
61324,cle_res_elec_05,01/01 05:00:00,N,Exist_Clear,1.620319,2.367461,0.0,0.0,0.042074,4.029854,...,0.0,0.0,0.042074,0.128326,0.000178,0.000083,0.0,0.0,0.000128,0.000389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96355,cle_res_elec_08,12/31 20:00:00,W,Exist_Clear,1.271018,0.314355,0.0,0.0,0.221847,1.807220,...,0.0,0.0,0.221847,0.271608,0.000140,0.000011,0.0,0.0,0.000673,0.000824
96356,cle_res_elec_08,12/31 21:00:00,W,Exist_Clear,1.271018,0.415513,0.0,0.0,0.224997,1.911528,...,0.0,0.0,0.224997,0.275932,0.000140,0.000015,0.0,0.0,0.000683,0.000837
96357,cle_res_elec_08,12/31 22:00:00,W,Exist_Clear,1.271018,0.475960,0.0,0.0,0.155698,1.902676,...,0.0,0.0,0.155698,0.207334,0.000140,0.000017,0.0,0.0,0.000472,0.000629
96358,cle_res_elec_08,12/31 23:00:00,W,Exist_Clear,1.271018,0.591899,0.0,0.0,0.086399,1.949316,...,0.0,0.0,0.086399,0.139380,0.000140,0.000021,0.0,0.0,0.000262,0.000423


In [37]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualDataResElec = clearAnnualDataResElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'C_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'C_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'C_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'C_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingElecCost($)':'C_HeatingElecCost($)',
 'HeatingGasCost($)':'C_HeatingGasCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingElecCost($/sf)':'C_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'C_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualDataResElec

In [38]:
# Reset index
allClearAnnualDataResElec.reset_index(inplace=True, drop=True)

# allClearAnnualDataResElec

In [39]:
# Define the scenario for the baselines.

loweAnnualDataResElec = allAnnualDataResElec[allAnnualDataResElec["GlazingSystem"] == "Exist_LowE"]

# loweAnnualDataResElec

In [40]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualDataResElec = loweAnnualDataResElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'L_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'L_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'L_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'L_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingElecCost($)':'L_HeatingElecCost($)',
 'HeatingGasCost($)':'L_HeatingGasCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingElecCost($/sf)':'L_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'L_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [41]:
# Reset index
allLoweAnnualDataResElec.reset_index(inplace=True, drop=True)

allLoweAnnualDataResElec

,L_Scenario,L_Date_Time,L_Elevation,L_GlazingSystem,L_FanEnergy(kBtu),L_HeatingElecEnergy(kBtu),L_HeatingGasEnergy(kBtu),L_CoolingEnergy(kBtu),L_LightingEnergy(kBtu),L_AnnualEnergy(kBtu),...,L_HeatingGasCost($),L_CoolingCost($),L_LightingCost($),L_AnnualCost($),L_FanCost($/sf),L_HeatingElecCost($/sf),L_HeatingGasCost($/sf),L_CoolingCost($/sf),L_LightingCost($/sf),L_AnnualCost($/sf)
0,cle_res_elec_10,01/01 01:00:00,E,Exist_LowE,0.677785,3.379772,0.0,0.0,0.015075,4.072632,...,0.0,0.0,0.015075,0.078872,0.000075,0.000119,0.0,0.0,0.000046,0.000239
1,cle_res_elec_10,01/01 02:00:00,E,Exist_LowE,0.677785,3.323989,0.0,0.0,0.015075,4.016848,...,0.0,0.0,0.015075,0.078225,0.000075,0.000117,0.0,0.0,0.000046,0.000237
2,cle_res_elec_10,01/01 03:00:00,E,Exist_LowE,0.677785,3.401684,0.0,0.0,0.015075,4.094543,...,0.0,0.0,0.015075,0.079126,0.000075,0.000120,0.0,0.0,0.000046,0.000240
3,cle_res_elec_10,01/01 04:00:00,E,Exist_LowE,0.677785,3.443431,0.0,0.0,0.015075,4.136290,...,0.0,0.0,0.015075,0.079610,0.000075,0.000121,0.0,0.0,0.000046,0.000242
4,cle_res_elec_10,01/01 05:00:00,E,Exist_LowE,0.677785,3.462645,0.0,0.0,0.042074,4.182504,...,0.0,0.0,0.042074,0.106833,0.000075,0.000122,0.0,0.0,0.000128,0.000324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,cle_res_elec_09,12/31 20:00:00,N,Exist_LowE,1.270645,1.224784,0.0,0.0,0.221847,2.717276,...,0.0,0.0,0.221847,0.282156,0.000140,0.000043,0.0,0.0,0.000673,0.000856
35036,cle_res_elec_09,12/31 21:00:00,N,Exist_LowE,1.270645,1.266865,0.0,0.0,0.224997,2.762507,...,0.0,0.0,0.224997,0.285794,0.000140,0.000045,0.0,0.0,0.000683,0.000867
35037,cle_res_elec_09,12/31 22:00:00,N,Exist_LowE,1.270645,1.277701,0.0,0.0,0.155698,2.704044,...,0.0,0.0,0.155698,0.216621,0.000140,0.000045,0.0,0.0,0.000472,0.000657
35038,cle_res_elec_09,12/31 23:00:00,N,Exist_LowE,1.270645,1.338347,0.0,0.0,0.086399,2.695391,...,0.0,0.0,0.086399,0.148025,0.000140,0.000047,0.0,0.0,0.000262,0.000449


In [42]:
# # Export individual CSV files for each condition

# allBaselineAnnualDataResElec.to_csv("Scraper_Output/cle_exist_all_annual_ress.csv", header=True, index=True)
# allClearAnnualDataResElec.to_csv("Scraper_Output/cle_clear_all_annual_res.csv", header=True, index=True)
# allLoweAnnualDataResElec.to_csv("Scraper_Output/cle_lowe_all_annual_res.csv", header=True, index=True)

---
## Merge the 3 dataframes to perform delta from existing calculation


In [43]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualDataResElec, allClearAnnualDataResElec, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualDataResElec, left_index=True, right_index=True)

AllMergedAnnualData

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_FanEnergy(kBtu),B_HeatingElecEnergy(kBtu),B_HeatingGasEnergy(kBtu),B_CoolingEnergy(kBtu),B_LightingEnergy(kBtu),B_AnnualEnergy(kBtu),...,L_HeatingGasCost($),L_CoolingCost($),L_LightingCost($),L_AnnualCost($),L_FanCost($/sf),L_HeatingElecCost($/sf),L_HeatingGasCost($/sf),L_CoolingCost($/sf),L_LightingCost($/sf),L_AnnualCost($/sf)
0,cle_res_elec_01,01/01 01:00:00,N,Exist,1.811737,4.531636,0.0,0.0,0.015075,6.358448,...,0.0,0.0,0.015075,0.078872,0.000075,0.000119,0.0,0.0,0.000046,0.000239
1,cle_res_elec_01,01/01 02:00:00,N,Exist,1.811737,4.382968,0.0,0.0,0.015075,6.209780,...,0.0,0.0,0.015075,0.078225,0.000075,0.000117,0.0,0.0,0.000046,0.000237
2,cle_res_elec_01,01/01 03:00:00,N,Exist,1.811737,4.527612,0.0,0.0,0.015075,6.354424,...,0.0,0.0,0.015075,0.079126,0.000075,0.000120,0.0,0.0,0.000046,0.000240
3,cle_res_elec_01,01/01 04:00:00,N,Exist,1.811737,4.577138,0.0,0.0,0.015075,6.403950,...,0.0,0.0,0.015075,0.079610,0.000075,0.000121,0.0,0.0,0.000046,0.000242
4,cle_res_elec_01,01/01 05:00:00,N,Exist,1.811737,4.600883,0.0,0.0,0.042074,6.454694,...,0.0,0.0,0.042074,0.106833,0.000075,0.000122,0.0,0.0,0.000128,0.000324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,cle_res_elec_04,12/31 20:00:00,W,Exist,1.462425,2.438205,0.0,0.0,0.221847,4.122477,...,0.0,0.0,0.221847,0.282156,0.000140,0.000043,0.0,0.0,0.000673,0.000856
35036,cle_res_elec_04,12/31 21:00:00,W,Exist,1.462425,2.603133,0.0,0.0,0.224997,4.290555,...,0.0,0.0,0.224997,0.285794,0.000140,0.000045,0.0,0.0,0.000683,0.000867
35037,cle_res_elec_04,12/31 22:00:00,W,Exist,1.462425,2.700461,0.0,0.0,0.155698,4.318584,...,0.0,0.0,0.155698,0.216621,0.000140,0.000045,0.0,0.0,0.000472,0.000657
35038,cle_res_elec_04,12/31 23:00:00,W,Exist,1.462425,2.803635,0.0,0.0,0.086399,4.352460,...,0.0,0.0,0.086399,0.148025,0.000140,0.000047,0.0,0.0,0.000262,0.000449


In [44]:
# list(AllMergedAnnualData.columns.values)

In [45]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingElecEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingElecEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingElecEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingGasEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingGasEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingGasEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingElecCost($)"] = AllMergedAnnualData["B_HeatingElecCost($)"] - AllMergedAnnualData["C_HeatingElecCost($)"]
AllMergedAnnualData["DeltaClear_HeatingGasCost($)"] = AllMergedAnnualData["B_HeatingGasCost($)"] - AllMergedAnnualData["C_HeatingGasCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingElecCost($/sf)"] = AllMergedAnnualData["B_HeatingElecCost($/sf)"] - AllMergedAnnualData["C_HeatingElecCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingGasCost($/sf)"] = AllMergedAnnualData["B_HeatingGasCost($/sf)"] - AllMergedAnnualData["C_HeatingGasCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [46]:
# list(AllMergedAnnualData.columns.values)

In [47]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingElecEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingElecEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingElecEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingGasEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingGasEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingGasEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingElecCost($)"] = AllMergedAnnualData["B_HeatingElecCost($)"] - AllMergedAnnualData["L_HeatingElecCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingGasCost($)"] = AllMergedAnnualData["B_HeatingGasCost($)"] - AllMergedAnnualData["L_HeatingGasCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingElecCost($/sf)"] = AllMergedAnnualData["B_HeatingElecCost($/sf)"] - AllMergedAnnualData["L_HeatingElecCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingGasCost($/sf)"] = AllMergedAnnualData["B_HeatingGasCost($/sf)"] - AllMergedAnnualData["L_HeatingGasCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [48]:
# list(AllMergedAnnualData.columns.values)

---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [49]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=True)

# AnnualDataDelta.sum().round(2)

In [50]:
AnnualDataDelta = AnnualDataDelta.sum().round(2)

In [51]:
# list(AnnualDataDelta.sum().round(2).columns.values)

In [52]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/cle_deltas_annual_res.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [53]:
# Copy dataframe to modify and leave original df intact
get_peak_ResElec = all_ResElec2.copy(deep=True)

# get_peak_ResElec

In [54]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalResPeak = (get_peak_ResElec["FanEnergy[J](Hourly)"] + get_peak_ResElec["HeatingElecEnergy[J](Hourly)"] + get_peak_ResElec["HeatingGasEnergy[J](Hourly)"] + get_peak_ResElec["CoolingEnergy[J](Hourly)"] + get_peak_ResElec["LightingEnergy[J](Hourly)"])
        
get_peak_ResElec["PeakEnergy[J](Hourly)"] = totalResPeak

get_peak_ResElec["PeakEnergy(w)"] = (get_peak_ResElec["PeakEnergy[J](Hourly)"] / 3600)

get_peak_ResElec["PeakEnergy(w/sf)"] = (get_peak_ResElec["PeakEnergy(w)"] / sf)

# get_peak_ResElec

In [55]:
# Get scenario and find the max
scenario1 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_01"]
scenario2 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_02"]
scenario3 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_03"]
scenario4 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_04"]
scenario5 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_05"]
scenario6 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_06"]
scenario7 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_07"]
scenario8 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_08"]
scenario9 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_09"]
scenario10 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_10"]
scenario11 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_11"]
scenario12 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_12"]

In [56]:
scenarioPeak1 = (scenario1[scenario1["PeakEnergy[J](Hourly)"] == scenario1["PeakEnergy[J](Hourly)"].max()])
scenarioPeak2 = (scenario2[scenario2["PeakEnergy[J](Hourly)"] == scenario2["PeakEnergy[J](Hourly)"].max()])
scenarioPeak3 = (scenario3[scenario3["PeakEnergy[J](Hourly)"] == scenario3["PeakEnergy[J](Hourly)"].max()])
scenarioPeak4 = (scenario4[scenario4["PeakEnergy[J](Hourly)"] == scenario4["PeakEnergy[J](Hourly)"].max()])
scenarioPeak5 = (scenario5[scenario5["PeakEnergy[J](Hourly)"] == scenario5["PeakEnergy[J](Hourly)"].max()])
scenarioPeak6 = (scenario6[scenario6["PeakEnergy[J](Hourly)"] == scenario6["PeakEnergy[J](Hourly)"].max()])
scenarioPeak7 = (scenario7[scenario7["PeakEnergy[J](Hourly)"] == scenario7["PeakEnergy[J](Hourly)"].max()])
scenarioPeak8 = (scenario8[scenario8["PeakEnergy[J](Hourly)"] == scenario8["PeakEnergy[J](Hourly)"].max()])
scenarioPeak9 = (scenario9[scenario9["PeakEnergy[J](Hourly)"] == scenario9["PeakEnergy[J](Hourly)"].max()])
scenarioPeak10 = (scenario10[scenario10["PeakEnergy[J](Hourly)"] == scenario10["PeakEnergy[J](Hourly)"].max()])
scenarioPeak11 = (scenario11[scenario11["PeakEnergy[J](Hourly)"] == scenario11["PeakEnergy[J](Hourly)"].max()])
scenarioPeak12 = (scenario12[scenario12["PeakEnergy[J](Hourly)"] == scenario12["PeakEnergy[J](Hourly)"].max()])

In [57]:
PeakDemandElec =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
27149,cle_res_elec_01,02/06 06:00:00,N,Exist,1.224249e+07,0,1.911483e+06,0.000,93529.28864,1.424751e+07,3957.640865,12.009227
35909,cle_res_elec_02,02/06 06:00:00,E,Exist,1.320179e+07,0,9.948133e+05,0.000,93529.28864,1.429013e+07,3969.481202,12.045156
44669,cle_res_elec_03,02/06 06:00:00,S,Exist,1.227821e+07,0,1.725646e+06,0.000,93529.28864,1.409738e+07,3915.939439,11.882687
53429,cle_res_elec_04,02/06 06:00:00,W,Exist,1.224175e+07,0,1.542940e+06,0.000,93529.28864,1.387822e+07,3855.060671,11.697954
66113,cle_res_elec_05,07/19 18:00:00,N,Exist_Clear,0.000000e+00,0,1.709527e+06,8882538.080,146228.53250,1.073829e+07,2982.859228,9.051310
70949,cle_res_elec_06,02/06 06:00:00,E,Exist_Clear,8.209287e+06,0,9.182772e+05,0.000,93529.28864,9.221093e+06,2561.414826,7.772462
83626,cle_res_elec_07,07/19 11:00:00,S,Exist_Clear,0.000000e+00,0,1.518963e+06,7888369.941,28248.69378,9.435581e+06,2620.994808,7.953254
93040,cle_res_elec_08,08/15 17:00:00,W,Exist_Clear,0.000000e+00,0,1.340995e+06,7403623.962,104211.56780,8.848830e+06,2458.008411,7.458681
97229,cle_res_elec_09,02/06 06:00:00,N,Exist_LowE,8.023403e+06,0,1.340602e+06,0.000,93529.28864,9.457534e+06,2627.092916,7.971758
869,cle_res_elec_10,02/06 06:00:00,E,Exist_LowE,8.679978e+06,0,7.151005e+05,0.000,93529.28864,9.488607e+06,2635.724297,7.997950


In [58]:
# Export dataframe  to csv
PeakDemandElec.to_csv("Scraper_Output/cle_peak_res.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files

In [59]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist"]

# baselinePeakDemandElec

In [60]:
# list(baselinePeakDemandElec.columns.values)

In [61]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'Elevation',
 'GlazingSystem':'GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'B_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'B_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'B_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'B_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'B_PeakEnergy(w/sf)'
                            })

# allbaselinePeakDemandElec

In [62]:
# Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

# allbaselinePeakDemandElec

In [63]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist_Clear"]

# clearPeakDemandElec

In [64]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'C_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'C_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'C_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'C_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'C_PeakEnergy(w/sf)'
                            })

# allClearPeakDemandElec

In [65]:
# Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [66]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist_LowE"]

# lowePeakDemandElec

In [67]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'L_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'L_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'L_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'L_PeakEnergy(w/sf)'
                            })

# allLowePeakDemandElec

---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [68]:
# merge existing and clear
MergedAnnualPeakData = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakData

#merge the above with lowe
AllMergedAnnualPeakData = pd.merge(MergedAnnualPeakData, allLowePeakDemandElec, left_index=True, right_index=True)

# AllMergedAnnualPeakData

In [69]:
# list(AllMergedAnnualPeakData.columns.values)

In [70]:
# Deltas of exist - clear

AllMergedAnnualPeakData["DeltaClear_PeakEnergy(w)"] = AllMergedAnnualPeakData["B_PeakEnergy(w)"] - AllMergedAnnualPeakData["C_PeakEnergy(w)"]
AllMergedAnnualPeakData["DeltaClear_PeakEnergy(w/sf)"] = AllMergedAnnualPeakData["B_PeakEnergy(w/sf)"] - AllMergedAnnualPeakData["C_PeakEnergy(w/sf)"]


In [71]:
# Deltas of exist - lowe

AllMergedAnnualPeakData["DeltaLowe_PeakEnergy(w)"] = AllMergedAnnualPeakData["B_PeakEnergy(w)"] - AllMergedAnnualPeakData["L_PeakEnergy(w)"]
AllMergedAnnualPeakData["DeltaLowe_PeakEnergy(w/sf)"] = AllMergedAnnualPeakData["B_PeakEnergy(w/sf)"] - AllMergedAnnualPeakData["L_PeakEnergy(w/sf)"]

# AllMergedAnnualPeakData

In [72]:
# list(AllMergedAnnualPeakData.columns.values)

In [73]:
# Export grouped file to csv
AllMergedAnnualPeakData.to_csv("Scraper_Output/cle_deltas_peak_res.csv", header=True, index=True)